In [1]:
//default_exp util

# Util

> Low level functions needed to build a neural net.

In [2]:
import {testEq} from './src/testutil.module.js'

In [3]:
/**
Returns the shape of an "n" dimentional array.
*/
function shape(m) {
    const result=[];
    while (Array.isArray(m)) {
        result.push(m.length);
        m=m[0];
    }
    return result;
}

In [4]:
testEq([0], shape([]));
testEq([1,0], shape([[]]));
testEq([1,1,3,2], shape([[[[0,1],[3,3],[4,4]]]]));

In [5]:
/**
Returns the transpose of a 2d array.
*/
function transpose(matrix) {
    const result = [];
    matrix.forEach(function(row,rowIndex) {
        row.forEach(function(elem,columnIndex) {
            if (rowIndex==0) {
                result[columnIndex]=[elem];
            } else {
                result[columnIndex].push(elem);
            }
        });
    });
    return result;
}

In [6]:
const left  = [
    [1,2,1],
    [0,1,0],
    [2,3,4]
];
const right = [
    [2,5],
    [6,7],
    [1,8]
];
testEq([[2,6,1],[5,7,8]],transpose(right));

In [7]:
/**
Returns the dot product of two 2d arrays.
*/
function dotProduct(a,b) {
    const bTransposed=transpose(b);
    const result=[];
    a.forEach(function(aRow,aRowIndex) {
        result[aRowIndex]=[];
        bTransposed.forEach(function(bRow) {
            const mults=[];
            aRow.forEach(function(aElem,aColumnIndex) {
                const bElem=bRow[aColumnIndex];
                mults.push(aElem*bElem);
            });
            result[aRowIndex].push(mults.reduce((a, b) => a + b, 0));
        });
    });
    return result;
}

In [8]:
let actual=dotProduct(left,right)
testEq([shape(left)[0],shape(right)[1]], shape(actual));
testEq([[15,27],[6,7],[26,63]], actual);
testEq([[2,5],[0,0],[4,10]], dotProduct([[1],[0],[2]],[[2,5]]));

In [9]:
/**
Returns a single value from a standard normal distribution.
*/
function randn_bm() {
    // Box-Muller transform - Max Collard - stack overflow
    var u=0, v=0;
    while(u==0) u=Math.random();
    while(v==0) v=Math.random();
    return Math.sqrt(-2.0*Math.log(u)) * Math.cos(2.0*Math.PI*v);
}

The following test checks that the mean of 100 `randn_bm` values is zero (after rounding to the nearest integer)

In [10]:
testEq(0, Math.round(Array(100).fill(0).map(_ => randn_bm()).reduce((a,b) => a+b)/100))

In [11]:
/**
Returns a 2d array filled with `randn_bm` values.
*/
function randn(d0,d1) {
    const result = [];
    for (let rowIndex = 0; rowIndex < d0; rowIndex++) {
        const row=[];
        result.push(row);
        for (let colIndex = 0; colIndex < d1; colIndex++) {
            row.push(randn_bm());
        }
    }
    return result;
}

In [12]:
let result=randn(20,5);
testEq([20,5],shape(result));
['Mean',result.map(row=>row.reduce((a,b)=>a+b)).reduce((a,b)=>a+b)/(20*5)];

[ 'Mean', -0.006723427276852156 ]


In [13]:
/**
Return a 1d or 2d array of zeros.
*/
function zeros(d0, d1) {
    if (d1 == null) {
        return new Array(d0).fill(0);
    }
    const result=[];
    for (let i=0; i<d0; i++) {
        result.push(new Array(d1).fill(0));
    }
    return result;
}

In [14]:
testEq([9],shape(zeros(9)));
testEq(0,zeros(9).reduce((a,b)=>a-b));
testEq([9,2],shape(zeros(9,2)));

In [15]:
/**
Elementwise sum of a 2d and a 1d matrix.
`shape(a2d)[1]` must equal `shape(b1d)`.
*/
function matrixSum(a2d,b1d) {
    return a2d.map(row => row.map((e, i) => e+b1d[i]));
}

In [16]:
testEq([[1,3],[10,30]], matrixSum([[0,1],[9,28]],[1,2]));

In [17]:
/**
Element wise subtraction of `b` from `a`, where a and b are 1d.
*/
function matrixSubtract1d(a,b) {
    return a.map((e,i) => e-b[i]);
}

/**
Element wise subtraction of `b` from `a`, where a and b are 2d.
*/
function matrixSubtract2d(a,b) {
    return a.map((row,i) => matrixSubtract1d(row,b[i]));
}

In [18]:
testEq([0,-1,-2,-3,-4],matrixSubtract1d([1,1,1,1,1],[1,2,3,4,5]));
testEq([[0,-1,-2,-3,-4],[0,1,2,3,4]],matrixSubtract2d([[1,1,1,1,1],[1,3,5,7,9]],[[1,2,3,4,5],[1,2,3,4,5]]));

In [19]:
/**
Return 1d array multiplied by a scalar value.
*/
function matrixMultiply1d(m,scalar) {
    return m.map(e => e*scalar);
}

/**
Return 2d array multiplied by a scalar value.
*/
function matrixMultiply2d(m,scalar) {
    return m.map(row => matrixMultiply1d(row,scalar));
}

In [20]:
testEq([10,20,30],matrixMultiply1d([1,2,3],10));
testEq([[10,20,30],[-10,-20,-30]],matrixMultiply2d([[1,2,3],[-1,-2,-3]],10));

In [21]:
/**
Return the index of the highest value in `a`.
*/
function argmax(a) {
    return a.indexOf(Math.max(...a));
}

In [22]:
testEq(2, argmax([0,0,1]));
testEq(2, argmax([0,0,.5]));
testEq(1, argmax([0,0.51,.5]));

In [23]:
export {
    shape,transpose,dotProduct,randn,zeros,
    matrixSum,matrixSubtract1d,matrixSubtract2d,matrixMultiply1d,matrixMultiply2d,argmax}